In [1]:
import os
from dotenv import load_dotenv
import json
import random
import matplotlib.pyplot as plt
import numpy as np
import asyncio

from benchmarklib import BenchmarkDatabase
from rbf import RandomBooleanFunctionTrial, RandomBooleanFunction
from benchmarklib.compilers import CompileType, XAGCompiler

from qiskit_ibm_runtime import QiskitRuntimeService

import logging
from typing import Iterable, List, Tuple, Dict, Any, Union, Optional
import qiskit
from qiskit.providers import Backend
from qiskit import QuantumCircuit, transpile
import random

from sqlalchemy import select, func

from tweedledum.bool_function_compiler import circuit_input, QuantumCircuitFunction
from tweedledum import BitVec

from benchmarklib import CompileType, BenchmarkDatabase
from benchmarklib import BatchQueue
from benchmarklib.compilers import SynthesisCompiler

In [2]:
load_dotenv()
API_TOKEN_OLD = os.getenv("API_TOKEN_OLD")
API_INSTANCE_OLD = os.getenv("API_INSTANCE_OLD")
service = QiskitRuntimeService()  # default service with new credentials
service_old = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    token=API_TOKEN_OLD,
    instance=API_INSTANCE_OLD
)

In [3]:
backend = service.backend("ibm_rensselaer")
# backend_fractional = service.backend("ibm_rensselaer", use_fractional_gates=True)  # ibm_rensselaer does not support fractional gates

In [4]:
backend.basis_gates

['ecr', 'id', 'rz', 'sx', 'x']

In [5]:
benchmark_db = BenchmarkDatabase("rbf.db", RandomBooleanFunction, RandomBooleanFunctionTrial)

### Compare input-output of transpilation

In [6]:
qc = QuantumCircuit(3, 3)
qc.ccx(0, 1, 2)
qc.measure(range(3), range(3))
qc.draw()

┌─┐      
q_0: ──■──┤M├──────
       │  └╥┘┌─┐   
q_1: ──■───╫─┤M├───
     ┌─┴─┐ ║ └╥┘┌─┐
q_2: ┤ X ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2

In [9]:
from qiskit import generate_preset_pass_manager


pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=1)
qc_transpiled = pass_manager.run(qc)
qc_transpiled.draw()

global phase: 13π/8
         ┌────┐┌──────────┐                                           ┌──────┐»
q_0 -> 6 ┤ √X ├┤ Rz(-π/2) ├───────────────────────────────────────────┤1     ├»
         ├───┬┘├─────────┬┘┌──────┐┌──────────┐┌────┐┌─────────┐┌────┐│  Ecr │»
q_2 -> 7 ┤ X ├─┤ Rz(π/2) ├─┤0     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/4) ├┤ √X ├┤0     ├»
         ├───┴┐├─────────┴┐│  Ecr │└──┬────┬──┘└────┘└─────────┘└────┘└──────┘»
q_1 -> 8 ┤ √X ├┤ Rz(-π/2) ├┤1     ├───┤ √X ├──────────────────────────────────»
         └────┘└──────────┘└──────┘   └────┘                                  »
    c: 3/═════════════════════════════════════════════════════════════════════»
                                                                              »
«            ┌────┐                                                     »
«q_0 -> 6 ───┤ √X ├─────────────────────────────────────────────────────»
«         ┌──┴────┴──┐┌────┐┌──────────┐┌────┐┌──────┐┌──────────┐┌────┐»
«q_2 -> 7 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π/4) ├┤ √X ├┤0     ├┤ Rz(-π/2) ├┤ √X ├»
«         └──────────┘└────┘└──────────┘└────┘│  Ecr │├──────────┤├────┤»
«q_1 -> 8 ────────────────────────────────────┤1     ├┤ Rz(-π/2) ├┤ √X ├»
«                                             └──────┘└──────────┘└────┘»
«    c: 3/══════════════════════════════════════════════════════════════»
«                                                                       »
«                           ┌──────┐   ┌────┐                           »
«q_0 -> 6 ──────────────────┤1     ├───┤ √X ├───────────────────────────»
«         ┌─────────┐ ┌────┐│  Ecr │┌──┴────┴─┐┌────┐┌───────────┐┌────┐»
«q_2 -> 7 ┤ Rz(π/4) ├─┤ √X ├┤0     ├┤ Rz(π/2) ├┤ √X ├┤ Rz(-3π/4) ├┤ √X ├»
«         ├─────────┴┐└────┘└──────┘└─────────┘└────┘└───────────┘└────┘»
«q_1 -> 8 ┤ Rz(3π/4) ├──────────────────────────────────────────────────»
«         └──────────┘                                                  »
«    c: 3/══════════════════════════════════════════════════════════════»
«                                                                       »
«                                                                             »
«q_0 -> 6 ────────────────────────────────────────────────────────────────────»
«         ┌──────┐┌──────────┐   ┌────┐   ┌──────┐┌─────────┐┌────┐           »
«q_2 -> 7 ┤0     ├┤ Rz(-π/2) ├───┤ √X ├───┤0     ├┤ Rz(π/2) ├┤ √X ├───────────»
«         │  Ecr │└──┬────┬──┘┌──┴────┴──┐│  Ecr │└┬────────┤├────┤┌─────────┐»
«q_1 -> 8 ┤1     ├───┤ √X ├───┤ Rz(-π/2) ├┤1     ├─┤ Rz(-π) ├┤ √X ├┤ Rz(π/2) ├»
«         └──────┘   └────┘   └──────────┘└──────┘ └────────┘└────┘└─────────┘»
«    c: 3/════════════════════════════════════════════════════════════════════»
«                                                                             »
«                                              ┌──────┐┌─────────┐ ┌────┐»
«q_0 -> 6 ─────────────────────────────────────┤1     ├┤ Rz(π/2) ├─┤ √X ├»
«         ┌──────┐┌──────────┐┌────┐┌─────────┐│  Ecr │├─────────┴┐├────┤»
«q_2 -> 7 ┤0     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├┤0     ├┤ Rz(-π/2) ├┤ √X ├»
«         │  Ecr │└───┬─┬────┘└────┘└─────────┘└──────┘└──────────┘└────┘»
«q_1 -> 8 ┤1     ├────┤M├────────────────────────────────────────────────»
«         └──────┘    └╥┘                                                »
«    c: 3/═════════════╩═════════════════════════════════════════════════»
«                      2                                                 »
«         ┌───────────┐┌────┐┌─────────┐┌──────┐┌──────────┐┌────┐┌─────────┐»
«q_0 -> 6 ┤ Rz(-3π/4) ├┤ √X ├┤ Rz(π/2) ├┤1     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├»
«         └┬─────────┬┘├────┤└─────────┘│  Ecr │├─────────┬┘├────┤├─────────┤»
«q_2 -> 7 ─┤ Rz(π/4) ├─┤ √X ├───────────┤0     ├┤ Rz(π/2) ├─┤ √X ├┤ Rz(π/2) ├»
«          └─────────┘ └────┘           └──────┘└─────────┘ └────┘└─────────┘»
«q_1 -> 8 ───────────────────────────────────────────────────────────────────»
«                                                          

### Experiment to compare Toffoli Implementations

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

pass_manager = generate_preset_pass_manager(backend=backend)
circuits = []
transpiled_circuits = []
jobs = []
for i in range(2**3):
    input_bits = [(i >> j) & 1 for j in range(3)]
    qc = QuantumCircuit(3, 3)
    for j in range(3):
        if input_bits[j]:
            qc.x(j)
    qc.ccx(0, 1, 2)
    qc.measure(range(3), range(3))
    circuits.append(qc)
    transpiled_circuits.append(pass_manager.run(qc))




[0, 0, 0]
[1, 0, 0]
[0, 1, 0]
[1, 1, 0]
[0, 0, 1]
[1, 0, 1]
[0, 1, 1]
[1, 1, 1]


### Backend Properties Analysis

In [ ]:
from benchmarklib.core.database import BackendPropertyManager
properties_db = BackendPropertyManager("rbf.db")

In [ ]:
import datetime
from datetime import timedelta

In [ ]:
gates = ['ecr', 'id', 'rz', 'sx', 'x']

start_date = datetime.date(2025, 1, 1)
end_date = datetime.utcnow().date()

date = datetime.combine(start_date, datetime.max.time())
end_date = datetime.combine(end_date, datetime.max.time())
while date <= end_date:
    properties = properties_db.latest(date)
    errors = properties.get_gate_errors()
    min_errors = {g: np.min(v) for g, v in errors.items()}
    max_errors = {g: np.max(v) for g, v in errors.items()}
    mean_errors = {g: np.mean(v) for g, v in errors.items()}
    median_errors = {g: np.median(v) for g, v in errors.items()}
    date += timedelta(days=1)
    
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import defaultdict
from datetime import datetime, timedelta, date

# Collect data
gates = ['ecr', 'id', 'sx', 'x']
dates = []
min_errors_over_time = defaultdict(list)
max_errors_over_time = defaultdict(list)
mean_errors_over_time = defaultdict(list)
median_errors_over_time = defaultdict(list)

start_date = date(2025, 1, 1)
end_date = datetime.now().date()

date = datetime.combine(start_date, datetime.max.time())
end_date = datetime.combine(end_date, datetime.max.time())

while date <= end_date:
    properties = properties_db.latest(date)
    errors = properties.get_gate_errors()
    
    dates.append(date)
    for gate in gates:
        if gate in errors:
            min_errors_over_time[gate].append(np.min(errors[gate]))
            max_errors_over_time[gate].append(np.max(errors[gate]))
            mean_errors_over_time[gate].append(np.mean(errors[gate]))
            median_errors_over_time[gate].append(np.median(errors[gate]))
    
    date += timedelta(days=1)

# Plot
fig, axes = plt.subplots(len(gates), 1, figsize=(12, 10), sharex=True)
fig.suptitle('IBM Rensselaer Gate Errors Over Time', fontsize=14, fontweight='bold')

for idx, gate in enumerate(gates):
    ax = axes[idx]
    
    # Shaded region for min-max range
    ax.fill_between(dates, 
                     min_errors_over_time[gate], 
                     max_errors_over_time[gate],
                     alpha=0.2, color='C0', label='Min-Max Range')
    
    # Mean and median lines
    ax.plot(dates, mean_errors_over_time[gate], 'o-', 
            label='Mean', markersize=3, linewidth=1.5, color='C0')
    ax.plot(dates, median_errors_over_time[gate], 's-', 
            label='Median', markersize=3, linewidth=1.5, color='C1')
    
    ax.set_ylabel(f'{gate.upper()}\nError Rate', fontsize=10)
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')

# Format x-axis
axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
axes[-1].xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
plt.xlabel('Date', fontsize=11)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Analyze Outliers

In [ ]:
outliers = [(13, 1)]
for num_vars, complexity in outliers:
    print(num_vars, complexity)
    trials = benchmark_db.find_trials(num_vars=num_vars, complexity=complexity, compiler_name="CLASSICAL_FUNCTION")
    statement_successes = {}
    statement_circuit_data = {}
    for trial in trials:
        if trial.problem.statement not in statement_successes:
            statement_successes[trial.problem.statement] = []
            statement_circuit_data[trial.problem.statement] = (trial.circuit_depth, trial.circuit_op_counts)
        statement_successes[trial.problem.statement].append(trial.calculate_success_rate())
    for key, value in statement_successes.items():
        print(value, key)
        print(statement_circuit_data[key])
    print("###")

In [ ]:
outliers = [(2, 15)]
for num_vars, complexity in outliers:
    print(num_vars, complexity)
    trials = benchmark_db.find_trials(num_vars=num_vars, complexity=complexity, compiler_name="XAG")
    statement_successes = {}
    statement_circuit_data = {}
    for trial in trials:
        if trial.problem.statement not in statement_successes:
            statement_successes[trial.problem.statement] = []
            statement_circuit_data[trial.problem.statement] = (trial.circuit_depth, trial.circuit_op_counts)
        statement_successes[trial.problem.statement].append(trial.calculate_success_rate())
    for key, value in statement_successes.items():
        print(value, key)
        print(statement_circuit_data[key])
    print("###")

### Classiq Test

In [ ]:
# TODO: authenticate with Classiq

from classiq import (
    Constraints,
    Output,
    Preferences,
    QArray,
    QNum,
    allocate,
    create_model,
    qfunc,
    synthesize,
)
@qfunc
def main(vertices: Output[QArray[QNum[1]]], oracle_result: Output[QNum[1]]):
    allocate(3, vertices)
    #allocate(1, oracle_result)
    oracle_result |= (vertices[0] & vertices[1]) | (vertices[1] & vertices[2]) | (vertices[0] & vertices[2])

oracle = synthesize(main)
print(oracle)

### Percent Circuit Metrics Collected

In [ ]:
from sqlalchemy import func, select
num_completed = benchmark_db.query(
    select(func.count(RandomBooleanFunctionTrial.job_id))
    .where(RandomBooleanFunctionTrial.circuit_depth != None)
    .distinct(RandomBooleanFunctionTrial.job_id)
)
print(f"Total completed jobs: {num_completed}")

total_jobs = benchmark_db.query(
    select(func.count(RandomBooleanFunctionTrial.job_id))
    .distinct(RandomBooleanFunctionTrial.job_id)
)
print(f"Total jobs: {total_jobs}")

### Basis Gate Success Comparisons

In [ ]:
from sqlalchemy.orm import joinedload
def plot_data(db, function_success_threshold):
    gate_percentages_data = {gate: {'%': [], 'success': []} for gate in ['ecr', 'rz', 'sx', 'x']}
    depth_data = []

    trials = db.query(
        select(RandomBooleanFunctionTrial)
        .where(RandomBooleanFunctionTrial.circuit_depth != None)
        .order_by(func.random())
        .limit(2000)
        .options(joinedload(RandomBooleanFunctionTrial.problem))
    )

    for trial in trials:
        depth_data.append(trial.circuit_depth)
        for gate in ['ecr', 'rz', 'sx', 'x']:
            gate_count = trial.circuit_op_counts.get(gate, 0)
            gate_percentage = gate_count / trial.circuit_num_gates if trial.circuit_num_gates > 0 else 0
            gate_percentages_data[gate]['%'].append(gate_percentage)
            gate_percentages_data[gate]['success'].append(1 if trial.calculate_success_rate() > function_success_threshold else 0)

    depth_data = np.array(depth_data, dtype=int)
    for gate, data in gate_percentages_data.items():
        percents = np.array(data['%'], dtype=float)
        successes = np.array(data['success'], dtype=int)
        plt.figure(figsize=(12, 8))
        plt.scatter(depth_data[successes == 1], percents[successes == 1], label=f"Success", color='green')
        plt.scatter(depth_data[successes == 0], percents[successes == 0], label=f"Fail", color='red')
        plt.xscale('log')
        plt.xlabel("Circuit Depth")
        plt.ylabel(f"Percentage {gate}")
        plt.title(f"{gate} Gate Influence on Success Rates")
        plt.legend()
        plt.savefig(f"gate_percentages_{gate}.png")
        plt.clf()

plot_data(benchmark_db, 0.5)

### Circuit Metrics

In [ ]:
from sqlalchemy import func, select
from sqlalchemy.orm import joinedload
trial = benchmark_db.query(
    select(RandomBooleanFunctionTrial)
    .join(RandomBooleanFunction)
    .where(RandomBooleanFunction.num_vars == 17, RandomBooleanFunction.complexity == 3, RandomBooleanFunctionTrial.compiler_name == XAGCompiler().name)
    .order_by(func.random()).limit(1)
    .options(joinedload(RandomBooleanFunctionTrial.problem))
)[0]
print(trial.__dict__)

In [ ]:
circuit = await trial.get_ibm_circuit(service)
trial.load_circuit_metrics(circuit)
print(trial.__dict__)

In [ ]:
job = service.job(trial.job_id)
circuit = job.inputs['pubs'][trial.job_pub_idx][0]
#circuit.draw('text')

In [ ]:
print(trial.circuit_op_counts)
print(trial.circuit_depth)
print(trial.circuit_num_qubits)
print(trial.circuit_num_gates)
print(trial.circuit_num_single_qubit_gates)
benchmark_db.save_trial(trial)


In [ ]:
print(circuit.count_ops())
print(circuit.depth())
print(circuit.width())
print(circuit.num_qubits)
print(circuit.size())
print(trial.calculate_success_rate())
print(trial.problem.statement)

In [ ]:
circuit.draw('mpl')

In [ ]:
classical_trial = benchmark_db.query(
    select(RandomBooleanFunctionTrial)
    .join(RandomBooleanFunction)
    .where(RandomBooleanFunction.statement == trial.problem.statement, RandomBooleanFunctionTrial.compiler_name == "CLASSICAL_FUNCTION")
    .order_by(func.random()).limit(1)
    .options(joinedload(RandomBooleanFunctionTrial.problem))
)[0]
classical_circuit = transpile(classical_trial.problem.oracle(compile_type="CLASSICAL_FUNCTION"), backend=backend)

In [ ]:
print(classical_circuit.count_ops())
print(classical_circuit.depth())
print(classical_circuit.width())
print(classical_circuit.num_qubits)
print(classical_circuit.size())
print(classical_trial.calculate_success_rate())

In [ ]:
trial.problem.oracle(compile_type="XAG").draw('mpl')

In [ ]:
count = 0
for trial in benchmark_db.find_trials(compiler_name="XAG"):
    if trial.counts is None:
        count += 1
        benchmark_db.delete_trial(trial.trial_id)
print(count)

### Benchmark Simulation

In [ ]:
from benchmarklib.runners.queue import simulate
def simulate_rbf_benchmark(db_manager: BenchmarkDatabase, compiler: "SynthesisCompiler", backend: Backend, num_vars_iter: Iterable[int], complexity_iter: Iterable[int], num_functions: int = 10, trials_per_instance: int = 5, shots: int = 10**3):
    """
    Run benchmarks for random boolean functions with varying number of variables and complexity.

    Args:
        db_manager: BenchmarkDatabase instance for storing results
        num_vars_iter: num_vars range
        complexity_iter: complexity range
        num_functions: Number of random functions to create trials for from each (num_vars, complexity) pair
        trials_per_instance: Number of trials (random input states) to run per problem instance (rbf function)
        shots: Number of shots to run per trial
    """
    #q = BatchQueue(db_manager, backend=backend, shots=shots, run_simulation=True)
    for num_vars in num_vars_iter:
        for complexity in complexity_iter:
            print(f"Simulating functions with num_vars={num_vars}, complexity={complexity}")
            problem_instances = db_manager.find_problem_instances(
                size_filters={'num_vars': num_vars, 'complexity': complexity},
                choose_untested=False,
                compiler=compiler,
                random_sample=True,
                limit=num_functions
            )
            for problem_instance in problem_instances:
                oracle = problem_instance.oracle(compiler.name)  # TODO: should update this to use new compiler API
                for _ in range(trials_per_instance):
                    input_state = ''.join(random.choice('01') for _ in range(num_vars))
                    qc = qiskit.QuantumCircuit(max(oracle.num_qubits, num_vars + 1), num_vars + 1)
                    for i, bit in enumerate(input_state):
                        if bit == '1':
                            qc.x(qc.qubits[i])

                    qc.compose(oracle, inplace=True)
                    qc.measure(range(num_vars + 1), range(num_vars + 1))

                    trial = RandomBooleanFunctionTrial(
                        problem_instance=problem_instance,
                        compiler_name=compiler.name,
                        input_state=input_state,
                    )
                    simulation_counts = simulate(qc)
                    if simulation_counts.get(trial.total_expected_results()) != 1000:
                        print(trial.total_expected_results(), simulation_counts, trial.__dict__)

    #q.finish_batch()

In [ ]:
simulate_rbf_benchmark(benchmark_db, XAGCompiler(), None, num_vars_iter=range(3, 6), complexity_iter=range(2, 3), num_functions=100, trials_per_instance=3, shots=1000)

In [ ]:
for trial in benchmark_db.find_trials(compiler_name="XAG", limit=10):
    print(trial.__dict__)
    print(trial._problem_instance.__dict__)
    print(trial.total_expected_results())
    print(trial.calculate_success_rate())